# PRACTICA-12
## DESCOMPOSICIÓN DE SERIES TEMPORALES

La descomposición de series temporales es una técnica que nos permite deconstruir una serie temporal en 
sus **partes componentes** individuales. Estas partes constan de hasta 4 componentes diferentes:

- 1) Componente de tendencia ($T$)

- 2) Componente estacional ($S$)

- 3) Componente cíclico ($C$)

- 4) Componente de ruido ($N$)

Nota: a veces agrupamos los componentes cíclico y de ruido y lo llamamos **Restante** o algo así.

No es necesario que una serie temporal en particular contenga los 4 componentes, puede que le 
falte un componente **estacional** o de **tendencia**.

## ¿De qué nos sirve este conocimiento? 

Bueno, predecir una serie de tiempo a menudo puede ser bastante difícil, pero si podemos descomponer 
la serie en componentes y tratar cada uno por separado, a veces podemos mejorar la predicción general. 
Cada componente tiene propiedades y comportamientos específicos y con este enfoque podemos utilizar métodos 
que se adaptan mejor a cada componente en particular.

- Vale la pena señalar que la descomposición se usa principalmente para ayudar a analizar y comprender series temporales históricas, pero también puede resultar útil cuando se intenta un **análisis prospectivo**.

Es útil pensar en los componentes como si se combinaran de forma **aditiva** o **multiplicativa**.

Un **modelo aditivo** sugiere que los componentes se suman de la siguiente manera:

$$y_{t}=T_{t}+S_{t}+C_{t}+N_{t}$$

mientras que un **modelo multiplicativo** sugiere que los componentes se multiplican juntos de la siguiente manera:

$$y_{t}=T_{t}\times S_{t}\times C_{t}\times N_{t}$$

- Un **modelo aditivo** es **lineal** donde los cambios a lo largo del tiempo se realizan consistentemente en la misma cantidad. 

- Un **modelo multiplicativo** es **no lineal**, como *cuadrático* o *exponencial* y los cambios aumentan o disminuyen con el tiempo.

- Una **tendencia lineal** es una línea recta, mientras que una **tendencia no lineal** es una línea curva.

- Una **estacionalidad lineal** tiene la *misma* frecuencia (ancho de ciclos) y amplitud (altura de ciclos), 
mientras que una **estacionalidad no lineal** tiene una frecuencia y/o amplitud creciente o decreciente a lo largo del tiempo.

Pueden existir **combinaciones** de los dos modelos, las tendencias pueden cambiar de dirección y los ciclos de naturaleza 
no repetitiva pueden *mezclarse* con los componentes de estacionalidad repetitivos. Básicamente, los datos del 
mundo real pueden ser confusos y no siempre siguen las reglas. Sin embargo, siguen siendo útiles para definir 
un marco simple desde el cual analizar nuestros datos.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
df = pd.read_csv('Ozono.csv')
df.head(10)

In [ ]:
df.set_index('Month', inplace=True)
df.index = pd.to_datetime(df.index)
df.head(10)

In [4]:
df.dropna(inplace=True)

In [ ]:
df.plot()

In [6]:
# Descomposición
result = seasonal_decompose(df['Ozon concentration'], 
                            model = 'Additive',
                           period = 12)

In [ ]:
result.seasonal.plot()

In [ ]:
result.trend.plot()

In [ ]:
result.plot()

## Descomposición de tendencia estacional usando LOESS (STL)

Haremos uso de STL para descomponer una serie de tiempo en tres componentes: **tendencia**, **estacional** y **residual**. 
STL utiliza LOESS (*suavizado de diagrama de dispersión estimado localmente*) para extraer estimaciones suavizadas 
de los tres componentes. Las entradas clave en STL son:

- season- La duración de la temporada más suave. Debe ser extraño.

- trend- La duración de la tendencia más suave, por lo general alrededor del 150% de *season*. Debe ser impar y más grande que *season*.

- low_pass- La longitud de la ventana de estimación de paso bajo, generalmente el número impar más pequeño mayor que la periodicidad de los datos.

Primero importamos los paquetes necesarios, preparamos el entorno de gráficos y preparamos los datos.

In [10]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pandas.plotting import register_matplotlib_converters

register_matplotlib_converters()
sns.set_style("darkgrid")

In [11]:
plt.rc("figure", figsize=(16, 12))
plt.rc("font", size=13)

La descomposición requiere 1 entrada, la *serie de datos*. Si la serie de datos no tiene una frecuencia, 
también debe especificar **period**. El valor predeterminado para *season* es 7, por lo que también debe 
cambiarse en la mayoría de las aplicaciones.

In [ ]:
from statsmodels.tsa.seasonal import STL

stl = STL(df, seasonal=13)
res = stl.fit()
fig = res.plot()

## Montaje robusto
La configuración **robust** utiliza una función de ponderación dependiente de los datos que vuelve a 
ponderar los datos al estimar el LOESS (y también se usa LOWESS). El uso de una estimación robusta permite que el 
modelo tolere errores más grandes que son visibles en el gráfico inferior.

- Aquí utilizamos una serie de medidas de la producción de equipos eléctricos en la UE.

In [17]:
from statsmodels.datasets import elec_equip as ds
elec_equip = ds.load().data

A continuación, estimamos el modelo con y sin *ponderación robusta*. La diferencia es menor y 
es más pronunciada durante la crisis financiera de 2008. La estimación no robusta asigna pesos 
iguales a todas las observaciones y, por lo tanto, produce errores más pequeños, en promedio. 
Los pesos varían entre 0 y 1.

In [ ]:
def add_stl_plot(fig, res, legend):
    """Agregue 3 parcelas de un segundo ajuste STL"""
    axs = fig.get_axes()
    comps = ["trend", "seasonal", "resid"]
    for ax, comp in zip(axs[1:], comps):
        series = getattr(res, comp)
        if comp == "resid":
            ax.plot(series, marker="o", linestyle="none")
        else:
            ax.plot(series)
            if comp == "trend":
                ax.legend(legend, frameon=False)


stl = STL(elec_equip, period=12, robust=True)
res_robust = stl.fit()
fig = res_robust.plot()
res_non_robust = STL(elec_equip, period=12, robust=False).fit()
add_stl_plot(fig, res_non_robust, ["Robust", "Non-robust"])

## grado LOESS
La configuración predeterminada estima el modelo LOESS con una constante y una tendencia. Esto se puede cambiar 
para incluir solo una constante al establecerlo COMPONENT_deg en 0. Aquí el grado hace poca diferencia, excepto en 
la tendencia en torno a la crisis financiera de 2008.

In [ ]:
stl = STL(elec_equip, period=12, seasonal_deg=0, trend_deg=0, low_pass_deg=0, robust=True)
res_deg_0 = stl.fit()
fig = res_robust.plot()
add_stl_plot(fig, res_deg_0, ["Grado 1", "Grado 0"])

## Actuación
Se pueden usar tres opciones para reducir el costo computacional de la descomposición STL:

- seasonal_jump

- trend_jump

- low_pass_jump

Cuando estos son distintos de cero, el LOESS para el componente COMPONENT solo se estima cada COMPONENT_jump observación, 
y se usa la interpolación lineal entre puntos. Estos valores normalmente no deberían ser más del 10-20% 
del tamaño de seasonal, trend o low_pass, respectivamente.

El siguiente ejemplo muestra cómo estos pueden reducir el costo computacional por un factor de 15 utilizando 
datos simulados con una tendencia cosinusoidal de baja frecuencia y un patrón estacional sinusoidal.

In [ ]:
import numpy as np

rs = np.random.RandomState(0xA4FD94BC)
tau = 2000
t = np.arange(tau)
period = int(0.05 * tau)
seasonal = period + ((period % 2) == 0)  # Ensure odd
e = 0.25 * rs.standard_normal(tau)
y = np.cos(t / tau * 2 * np.pi) + 0.25 * np.sin(t / period * 2 * np.pi) + e
plt.plot(y)
plt.title("Datos simulados")
xlim = plt.gca().set_xlim(0, tau)

Primero, el modelo de línea base se estima con todos los saltos iguales a 1.

In [ ]:
mod = STL(y, period=period, seasonal=seasonal)
%timeit mod.fit()
res = mod.fit()
fig = res.plot(observed=False, resid=False)

Todos los saltos están configurados al 15% de la longitud de su ventana. La interpolación lineal limitada hace 
poca diferencia en el ajuste del modelo.

In [ ]:
low_pass_jump = seasonal_jump = int(0.15 * (period + 1))
trend_jump = int(0.15 * 1.5 * (period + 1))
mod = STL(
    y,
    period=period,
    seasonal=seasonal,
    seasonal_jump=seasonal_jump,
    trend_jump=trend_jump,
    low_pass_jump=low_pass_jump,
)
%timeit mod.fit()
res = mod.fit()
fig = res.plot(observed=False, resid=False)

## Información de los precios de las acciones de Apple Inc. (AAPL)

In [ ]:
# importando modulos necesarios
!pip install pandas-datareader

import numpy as np
import pandas as pd
import datetime as dt
import statsmodels.api as sm 

# librerías de visualizaciones
import seaborn as sns
import matplotlib.pyplot as plt 


# graficos incrustados
%matplotlib inline

In [ ]:
# Creando una serie de tiempo de Acciones Apple Inc. desde yahoo finance
from pandas_datareader import data
wft = data.DataReader("AAPL", 'yahoo', '2016-1-1', '2016-9-30')
wft.head(5)

In [ ]:
# filtrando sólo del 2016-02-04 al 2016-02-18
wft['2016-02-04':'2016-02-18']

In [ ]:
# valores al 2016-02-16
wft.loc['2016-2-16']

In [ ]:
# valor de la columna Adj Close al 2016-2-16
wft['Adj Close']['2016-2-16']

In [ ]:
# filtrando todo febrero de 2016
wft['2016-2']

In [ ]:
# Valores al cierre de cada mes.
wft.asfreq('M', method='ffill')

In [ ]:
# Valores al cierre de cada mes (días laborales).
wft.asfreq('BM')

In [ ]:
# valores al cierre de cada trimestre
wft.asfreq('BQ')

Una operación común en los datos de series de tiempo es desplazar los valores hacia 
atrás y adelante en el tiempo, como por ejemplo para calcular el cambio porcentual 
de una muestra a otra.

In [ ]:
# desplazando el 1 dia el valor de cierre
desplazado = wft['Adj Close'].shift(1)
desplazado[:5]

In [ ]:
# calculando el porcentaje de variación del día.
variacion_diaria = wft['Adj Close'] / wft['Adj Close'].shift(1) - 1
wft['var_diaria'] = variacion_diaria
wft['var_diaria'][:5]

In [ ]:
# mismo resultado utilizando pct_change()
wft['Adj Close'].pct_change()[:5]

In [ ]:
# calculando rendimiento acumulado diario
rendimiento_diario = (1 + wft['Adj Close'].pct_change()).cumprod()
wft['rend_diario'] = rendimiento_diario
wft['rend_diario'][:5]

Una operación fundamental para entender el comportamiento de una serie de tiempo y 
poder determinar si se trata de una serie estacionaria o no; es realizar gráficos 
de la misma.

In [ ]:
# graficando Adj Close
plot = wft['Adj Close'].plot(figsize=(10, 8))

In [22]:
# Aplicando el filtro Hodrick-Prescott para separar en tendencia y 
# componente ciclico.
wft_ciclo, wft_tend = sm.tsa.filters.hpfilter(wft['Adj Close'])
wft['tend'] = wft_tend

In [ ]:
# graficando la variacion del precio real con la tendencia.
wft[['Adj Close', 'tend']].plot(figsize=(10, 8), fontsize=12);
legend = plt.legend()
legend.prop.set_size(14);

In [ ]:
# graficando rendimiento diario
plot = wft['var_diaria'].plot(figsize=(10, 8))

## EJERCICIO
Descomponer la serie de tiempo ('Adj Close') en su tendencia, su estacionalidad y sus factores residuales.